In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
indices = ['^GSPTSE', '^BSESN', '^GDAXI', '^FCHI', '^FTSE']
equities = ['JNJ', 'NVDA', 'MSFT', 'JPM', 'TSLA']
indexdata=yf.download(indices, start='2010-01-01',end='2023-05-02',progress=False)['Adj Close']
#here I am defining that equity is the array from which I need to extract the data and the below code will help me do the same
equitydata=yf.download(equities, start='2010-01-01',end='2023-05-02',progress=False)['Adj Close']
#it was told to define the risk free rate as 0.0525
riskfreerate= 0.0525
#I therefore need to solve the problem for finding the daily returns , cumulative returns,mdd, sortino ration , volatility and sortino ratio
#the code for daily returns is as follow
#this is for the equity
for equity in equities:
    data=equitydata[equity]
    dailyreturns=(data-data.shift(1))/(data.shift(1))
    cumulativereturn=(1+dailyreturns).cumprod()-1
    print(f"Daily returns for {equity}:")
    print(dailyreturns)
    print(f"cumulative return for {equity}:" )
    print(cumulativereturn)

for index in indices:
    data=indexdata[index]
    dailyreturns=(data-data.shift(1))/(data.shift(1))
    cumulativereturn=(1+dailyreturns).cumprod()-1
    print(f"Daily returns for {index}:")
    print(dailyreturns)
    print(f"cumulative return for {index}:" )
    print(cumulativereturn)
#the cumulative return code ran well 
def calculatemdd(data):
    peak = data[0]
    mdd = 0
    for i in range(1, len(data)):
        if data[i] > peak:
            peak = data[i]
        else:
            drawdown = (data[i] - peak) / peak
            mdd = min(mdd, drawdown)
            peak = data[i]  # Update peak for next potential drawdown
    return mdd

print("MDD for Equities:")
for equity in equities:
    data = equitydata[equity]
    dailyreturns = (data - data.shift(1)) / data.shift(1)
    cumulativereturns = (1 + dailyreturns).cumprod() - 1
    mdd = calculatemdd(cumulativereturns)
    print(f"{equity}: {mdd}")
print("MDD for Indices:")
for index in indices:
    data = indexdata[index]
    dailyreturns = (data - data.shift(1)) / data.shift(1)
    cumulativereturns = (1 + dailyreturns).cumprod() - 1
    mdd = calculatemdd(cumulativereturns)
    print(f"{index}: {mdd}")
#the code for MDD worked well!
#now I need to work for the code for sharpe ratio
for equity in equities:
    data = equitydata[equity]
    validdata = data.dropna().replace(0, np.nan)#here I need to remove the points where I am getting 0 as the data point
    dailyreturns = (validdata - validdata.shift(1)) / validdata.shift(1)
    cumulativereturns = (1 + dailyreturns).cumprod() - 1
    mdd = calculatemdd(cumulativereturns)
    mean = sum(dailyreturns.dropna()) / len(dailyreturns.dropna())
    sumofsquare = [(a - mean) ** 2 for a in dailyreturns if not np.isnan(a)]
    if len(sumofsquare) > 0:
        standarddeviation = math.sqrt(sum(sumofsquare) / len(sumofsquare))
    else:
        standarddeviation = 0.0 
    if standarddeviation != 0:
        sharperatio = (mean - riskfreerate) / standarddeviation
    else:
        sharperatio = 0.0  

    print(f"The Sharpe ratio of {equity} is:")
    print(sharperatio)
for index in indices:
    data = indexdata[index]
    validdata = data.dropna().replace(0, np.nan)#here I need to remove the points where I am getting 0 as the data point
    dailyreturns = (validdata - validdata.shift(1)) / validdata.shift(1)
    cumulativereturns = (1 + dailyreturns).cumprod() - 1
    mdd = calculatemdd(cumulativereturns)
    mean = sum(dailyreturns.dropna()) / len(dailyreturns.dropna())
    sumofsquare = [(a - mean) ** 2 for a in dailyreturns if not np.isnan(a)]
    if len(sumofsquare) > 0:
        standarddeviation = (math.sqrt(sum(sumofsquare) / len(sumofsquare)))*math.sqrt(252)
    else:
        standarddeviation = 0.0 
    if standarddeviation != 0:
        sharperatio = ((mean - riskfreerate) / standarddeviation)*math.sqrt(252)
    else:
        sharperatio = 0.0  

    print(f"The Sharpe ratio of {index} is:")
    print(sharperatio)
#the above code is working to calculate the sharpe ratio
#and now there goes the code for sortino ratio
for equity in equities:
    data = equitydata[equity]
    validdata = data.dropna().replace(0, np.nan)
    dailyreturns = (validdata - validdata.shift(1)) / validdata.shift(1)
    cumulativereturns = (1 + dailyreturns).cumprod() - 1
    mdd = calculatemdd(cumulativereturns)
    mean = sum(dailyreturns.dropna()) / len(dailyreturns.dropna())
    negativereturns = dailyreturns[dailyreturns < 0]
    downsidedeviation = math.sqrt(sum((negativereturns - mean) ** 2) / len(negativereturns))
    if downsidedeviation != 0:
        sortinoratio = (mean - riskfreerate) / downsidedeviation
    else:
        sortinoratio = 0.0  

    print(f"The Sortino ratio of {equity} is:")
    print(sortinoratio)
for index in indices:
    data = indexdata[index]
    validdata = data.dropna().replace(0, np.nan)
    dailyreturns = (validdata - validdata.shift(1)) / validdata.shift(1)
    cumulativereturns = (1 + dailyreturns).cumprod() - 1
    mdd = calculatemdd(cumulativereturns)
    mean = sum(dailyreturns.dropna()) / len(dailyreturns.dropna())
    negativereturns = dailyreturns[dailyreturns < 0]
    downsidedeviation = math.sqrt(sum((negativereturns - mean) ** 2) / len(negativereturns))
    if downsidedeviation != 0:
        sortinoratio = (mean - riskfreerate) / downsidedeviation
    else:
        sortinoratio = 0.0  

    print(f"The Sortino ratio of {index} is:")
    print(sortinoratio)
#the above calculation was successful 
#volatility calculation 
for equity in equities:
    data = equitydata[equity]
    validdata = data.dropna().replace(0, np.nan)
    dailyreturns = (validdata - validdata.shift(1)) / validdata.shift(1)
    
    # Calculate volatility as the standard deviation of daily returns
    volatility = math.sqrt(sum((dailyreturns - dailyreturns.mean()) ** 2) / len(dailyreturns))

    print(f"The volatility of {equity} is:")
    print(volatility)



Daily returns for JNJ:
Date
2010-01-04         NaN
2010-01-05   -0.011595
2010-01-06    0.008134
2010-01-07   -0.007137
2010-01-08    0.003438
                ...   
2023-04-25    0.009164
2023-04-26   -0.015498
2023-04-27    0.002337
2023-04-28    0.004294
2023-05-01   -0.000611
Name: JNJ, Length: 3354, dtype: float64
cumulative return for JNJ:
Date
2010-01-04         NaN
2010-01-05   -0.011595
2010-01-06   -0.003556
2010-01-07   -0.010668
2010-01-08   -0.007266
                ...   
2023-04-25    2.757308
2023-04-26    2.699077
2023-04-27    2.707721
2023-04-28    2.723643
2023-05-01    2.721369
Name: JNJ, Length: 3354, dtype: float64
Daily returns for NVDA:
Date
2010-01-04         NaN
2010-01-05    0.014602
2010-01-06    0.006397
2010-01-07   -0.019597
2010-01-08    0.002161
                ...   
2023-04-25   -0.029621
2023-04-26    0.027247
2023-04-27    0.010016
2023-04-28    0.019210
2023-05-01    0.041839
Name: NVDA, Length: 3354, dtype: float64
cumulative return for NVDA:
Dat